# Introduction

In this project, we will focus on credit modelling, a well known data science problem that focuses on modeling a borrower's credit risk. Credit has played a key role in the economy for centuries and some form of credit has existed since the beginning of commerce. We'll be working with financial lending data from Lending Club. [Lending Club](https://www.lendingclub.com/auth/login?login_url=%2Fstatistics%2Fadditional-statistics%3F) is a marketplace for personal loans that matches borrowers who are seeking a loan with investors looking to lend money and make a return. You can read more about their marketplace [here](https://www.lendingclub.com/public/how-peer-lending-works.action).

We are going to build a machine learning model that can accurately predict if a borrower will pay off their loan on time or not. The data we are going to work with is approved loans data from 2007 to 2011, since a good number of the loans have already finished. In the datasets for later years, many of the loans are current and still being paid off. The size of the dataset has been reduced to make it easier to work with, by:
- removing the `desc` column which contains a long text explanation for each loan
- removing the `url` column which contains a link to each loan on Lending Club which can only be accessed with an investor account
- removing all columns containing more than 50% missing values

# Exporing and Cleaning the Data

In [8]:
import pandas as pd

loans_2007 = pd.read_csv("loans_2007.csv")
print(loans_2007.iloc[0])
print(len(loans_2007.columns))

id                                1077501
member_id                      1.2966e+06
loan_amnt                            5000
funded_amnt                          5000
funded_amnt_inv                      4975
term                            36 months
int_rate                           10.65%
installment                        162.87
grade                                   B
sub_grade                              B2
emp_title                             NaN
emp_length                      10+ years
home_ownership                       RENT
annual_inc                          24000
verification_status              Verified
issue_d                          Dec-2011
loan_status                    Fully Paid
pymnt_plan                              n
purpose                       credit_card
title                            Computer
zip_code                            860xx
addr_state                             AZ
dti                                 27.65
delinq_2yrs                       

The Dataframe contains many columns and can be cumbersome to try to explore all at once. Let's break up the columns into 3 groups of 18 columns and use the [data dictionary](https://docs.google.com/spreadsheets/d/191B2yJ4H1ZPXq0_ByhUgWMFZOYem5jFz0Y3by_7YBY4/edit#gid=2081333097) to become familiar with what each column represents.

At the moment, let's focus on just columns that we need to remove from consideration. Then, we can circle back and further dissect the columns we decided to keep. We created a table that contains the name, data type, first row's value, and description from the data dictionary for the first 18 rows.

|name	|dtype	|first value	|description|
|---|---|---|---|
|id	|object	|1077501	|A unique LC assigned ID for the loan listing.|
|member_id	|float64	|1.2966e+06	|A unique LC assigned Id for the borrower member.|
|loan_amnt	|float64	|5000	|The listed amount of the loan applied for by the borrower.|
|funded_amnt	|float64	|5000	|The total amount committed to that loan at that point in time.|
|funded_amnt_inv	|float64	|49750	|The total amount committed by investors for that loan at that point in time.|
|term	|object	|36 months	|The number of payments on the loan. Values are in months and can be either 36 or 60.|
|int_rate	|object	|10.65%	|Interest Rate on the loan|
|installment	|float64	|162.87	|The monthly payment owed by the borrower if the loan originates.|
|grade	|object	|B	|LC assigned loan grade|
|sub_grade	|object	|B2	|LC assigned loan subgrade|
|emp_title	|object	|NaN	|The job title supplied by the Borrower when applying for the loan.|
|emp_length	|object	|10+ years	|Employment length in years. Possible values are between 0 and 10 where 0 means less than one year and 10 means ten or more years.|
|home_ownership	|object	|RENT	|The home ownership status provided by the borrower during registration. Our values are: RENT, OWN, MORTGAGE, OTHER.|
|annual_inc	|float64	|24000	|The self-reported annual income provided by the borrower during registration.|
|verification_status	|object	|Verified	|Indicates if income was verified by LC, not verified, or if the income source was verified|
|issue_d	|object	|Dec-2011	|The month which the loan was funded|
|loan_status	|object	|Charged Off	|Current status of the loan|
|pymnt_plan	|object	|n	|Indicates if a payment plan has been put in place for the loan|
|purpose	|object	|car	|A category provided by the borrower for the loan request.|

After analyzing each column, we can conclude that the following features need to be removed:

- `id`: randomly generated field by Lending Club for unique identification purposes only
- `member_id`: also a randomly generated field by Lending Club for unique identification purposes only
- `funded_amnt`: leaks data from the future (after the loan is already started to be funded)
- `funded_amnt_inv`: also leaks data from the future (after the loan is already started to be funded)
- `grade`: contains redundant information as the interest rate column (`int_rate`)
- `sub_grade`: also contains redundant information as the interest rate column (`int_rate`)
- `emp_title`: requires other data and a lot of processing to potentially be useful
- `issue_d`: leaks data from the future (after the loan is already completely funded)

In [9]:
loans_2007 = loans_2007.drop(["id", "member_id", "funded_amnt", "funded_amnt_inv", "grade", "sub_grade", "emp_title", "issue_d"], axis=1)

Let's now look at the next 18 columns:

|name	|dtype	|first value	|description|
|---|---|---|---|
|title	|object	|Computer	|The loan title provided by the borrower|
|zip_code	|object	|860xx	|The first 3 numbers of the zip code provided by the borrower in the loan application.|
|addr_state	|object	|AZ	|The state provided by the borrower in the loan application|
|dti	|float64	|27.65	|A ratio calculated using the borrower’s total monthly debt payments on the total debt obligations, excluding mortgage and the requested LC loan, divided by the borrower’s self-reported monthly income.|
|delinq_2yrs	|float64	|0	|The number of 30+ days past-due incidences of delinquency in the borrower's credit file for the past 2 years|
|earliest_cr_line	|object	|Jan-1985	|The month the borrower's earliest reported credit line was opened|
|inq_last_6mths	|float64	|1	|The number of inquiries in past 6 months (excluding auto and mortgage inquiries)|
|open_acc	|float64	|3	|The number of open credit lines in the borrower's credit file.|
|pub_rec	|float64	|0	|Number of derogatory public records|
|revol_bal	|float64	|13648	|Total credit revolving balance|
|revol_util	|object	|83.7%	|Revolving line utilization rate, or the amount of credit the borrower is using relative to all available revolving credit.|
|total_acc	|float64	|9	|The total number of credit lines currently in the borrower's credit file|
|initial_list_status	|object	|f	|The initial listing status of the loan. Possible values are – W, F|
|out_prncp	|float64	|0	|Remaining outstanding principal for total amount funded|
|out_prncp_inv	|float64	|0	|Remaining outstanding principal for portion of total amount funded by investors|
|total_pymnt	|float64	|5863.16	|Payments received to date for total amount funded|
|total_pymnt_inv	|float64	|5833.84	|Payments received to date for portion of total amount funded by investors|
|total_rec_prncp	|float64	|5000	|Principal received to date|

Within this group of columns, we need to drop the following columns:

- `zip_code`: redundant with the addr_state column since only the first 3 digits of the 5 digit zip code are visible (which only can be used to identify the state the borrower lives in)
- `out_prncp`: leaks data from the future, (after the loan already started to be paid off)
- `out_prncp_inv`: also leaks data from the future, (after the loan already started to be paid off)
- `total_pymnt`: also leaks data from the future, (after the loan already started to be paid off)
- `total_pymnt_inv`: also leaks data from the future, (after the loan already started to be paid off)
- `total_rec_prncp`: also leaks data from the future, (after the loan already started to be paid off)

In [10]:
loans_2007 = loans_2007.drop(["zip_code", "out_prncp", "out_prncp_inv", "total_pymnt", "total_pymnt_inv", "total_rec_prncp"], axis=1)

Let's now move on to the last group of features:

|name	|dtype	|first value	|description|
|---|---|---|---|
|total_rec_int	|float64	|863.16	|Interest received to date|
|total_rec_late_fee	|float64	|0	|Late fees received to date|
|recoveries	|float64	|0	|post charge off gross recovery|
|collection_recovery_fee	|float64	|0	|post charge off collection fee|
|last_pymnt_d	|object	|Jan-2015	|Last month payment was received|
|last_pymnt_amnt	|float64	|171.62	|Last total payment amount received|
|last_credit_pull_d	|object	|Jun-2016	|The most recent month LC pulled credit for this loan|
|collections_12_mths_ex_med	|float64	|0	|Number of collections in 12 months excluding medical collections|
|policy_code	|float64	|1	|publicly available policy_code=1 new products not publicly available policy_code=2|
|application_type	|object	|INDIVIDUAL	|Indicates whether the loan is an individual application or a joint application with two co-borrowers|
|acc_now_delinq	|float64	|0	|The number of accounts on which the borrower is now delinquent.|
|chargeoff_within_12_mths	|float64	|0	|Number of charge-offs within 12 months|
|delinq_amnt	|float64	|0	|The past-due amount owed for the accounts on which the borrower is now delinquent.|
|pub_rec_bankruptcies	|float64	|0	|Number of public record bankruptcies|
|tax_liens	|float64	|0	|Number of tax liens|

In the last group of columns, we need to drop the following columns:

- `total_rec_int`: leaks data from the future, (after the loan already started to be paid off),
- `total_rec_late_fee`: also leaks data from the future, (after the loan already started to be paid off),
- `recoveries`: also leaks data from the future, (after the loan already started to be paid off),
- `collection_recovery_fee`: also leaks data from the future, (after the loan already started to be paid off),
- `last_pymnt_d`: also leaks data from the future, (after the loan already started to be paid off),
- `last_pymnt_amnt`: also leaks data from the future, (after the loan already started to be paid off).

In [11]:
loans_2007 = loans_2007.drop(["total_rec_int", "total_rec_late_fee", "recoveries", "collection_recovery_fee", "last_pymnt_d", "last_pymnt_amnt"], axis=1)
print(loans_2007.iloc[0])
print(loans_2007.shape[1])

loan_amnt                            5000
term                            36 months
int_rate                           10.65%
installment                        162.87
emp_length                      10+ years
home_ownership                       RENT
annual_inc                          24000
verification_status              Verified
loan_status                    Fully Paid
pymnt_plan                              n
purpose                       credit_card
title                            Computer
addr_state                             AZ
dti                                 27.65
delinq_2yrs                             0
earliest_cr_line                 Jan-1985
inq_last_6mths                          1
open_acc                                3
pub_rec                                 0
revol_bal                           13648
revol_util                          83.7%
total_acc                               9
initial_list_status                     f
last_credit_pull_d               J

Just by becoming familiar with the columns in the dataset, we were able to reduce the number of columns from 52 to 32 columns. We now need to decide on a target column that we want to use for modeling.

We should use the `loan_status` column, since it's the only column that directly describes if a loan was paid off on time, had delayed payments, or was defaulted on the borrower. Currently, this column contains text values and we need to convert it to a numerical one for training a model.

In [12]:
print(loans_2007["loan_status"].value_counts(dropna=False))

Fully Paid                                             33136
Charged Off                                             5634
Does not meet the credit policy. Status:Fully Paid      1988
Current                                                  961
Does not meet the credit policy. Status:Charged Off      761
Late (31-120 days)                                        24
In Grace Period                                           20
Late (16-30 days)                                          8
Default                                                    3
NaN                                                        3
Name: loan_status, dtype: int64


There are 8 different possible values for the `loan_status` column. Their meanings are explained below:

- Fully Paid: Loan has been fully paid off.
- Charged Off: Loan for which there is no longer a reasonable expectation of further payments.
- Does not meet the credit policy. Status:Fully Paid: While the loan was paid off, the loan application today would no longer meet the credit policy and wouldn't be approved on to the marketplace.
- Does not meet the credit policy. Status:Charged Off: While the loan was charged off, the loan application today would no longer meet the credit policy and wouldn't be approved on to the marketplace.
- In Grace Period: The loan is past due but still in the grace period of 15 days.
- Late (16-30 days): Loan hasn't been paid in 16 to 30 days (late on the current payment).
- Late (31-120 days): Loan hasn't been paid in 31 to 120 days (late on the current payment).
- Current: Loan is up to date on current payments.
- Default: Loan is defaulted on and no payment has been made for more than 121 days.

From the investor's perspective, we're interested in trying to predict which loans will be paid off on time and which ones won't be. Only the `Fully Paid` and `Charged Off` values describe the final outcome of the loan. The other values describe loans that are still ongoing and where the jury is still out on if the borrower will pay back the loan on time or not. Since we're interested in being able to predict which of these 2 values a loan will fall under, we can treat the problem as a binary classification one. Let's remove all the loans that don't contain either `Fully Paid` and `Charged Off` as the loan's status and then transform the `Fully Paid` values to 1 for the positive case and the `Charged Off` values to 0 for the negative case. 

In [13]:
loans_2007 = loans_2007[(loans_2007["loan_status"] == "Fully Paid") | (loans_2007["loan_status"] == "Charged Off")]

mapping_dict = {"loan_status":{"Fully Paid":1, "Charged Off":0}}

loans_2007 = loans_2007.replace(mapping_dict)

To wrap up this section, let's look for any columns that contain only one unique value and remove them. These columns won't be useful for the model since they don't add any information to each loan application. In addition, removing these columns will reduce the number of columns we'll need to explore further.

In [14]:
drop_columns = []

for column in loans_2007.columns:
    loans_2007[column] = loans_2007[column].dropna()
    if loans_2007[column].nunique() == 1:
        drop_columns.append(column)
        
loans = loans_2007.drop(drop_columns, axis=1)
print(drop_columns)

['pymnt_plan', 'initial_list_status', 'collections_12_mths_ex_med', 'policy_code', 'application_type', 'acc_now_delinq', 'chargeoff_within_12_mths', 'delinq_amnt', 'tax_liens']


It looks we were able to remove 9 more columns since they only contained 1 unique value.

In this section, we started to become familiar with the columns in the dataset and removed many columns that aren't useful for modeling. We also selected our target column and decided to focus our modeling efforts on binary classification. In the next section, we'll explore the individual features in greater depth and work towards training our machine learning model.

# Preparing the Features

Let's start by computing the number of missing values and come up with a strategy for handling them. Then, we'll focus on the categorical columns.

In [15]:
null_counts = loans.isnull().sum()

print(null_counts[null_counts > 0])

emp_length              1036
title                     11
revol_util                50
last_credit_pull_d         2
pub_rec_bankruptcies     697
dtype: int64


While most of the columns have no missing values, two columns have fifty or less rows with missing values, and two columns, `emp_length` and `pub_rec_bankruptcies`, contain a relatively high amount of missing values.

Domain knowledge tells us that employment length is frequently used in assessing how risky a potential borrower is, so we'll keep this column despite its relatively large amount of missing values.

Let's inspect the values of the column `pub_rec_bankruptcies`.

In [16]:
print(loans["pub_rec_bankruptcies"].value_counts(normalize=True, dropna=False))

0.0    0.939438
1.0    0.042456
NaN    0.017978
2.0    0.000129
Name: pub_rec_bankruptcies, dtype: float64


We see that this column offers very little variability, nearly 94% of values are in the same category. It probably won't have much predictive value. Let's drop it. In addition, we'll remove the remaining rows containing null values.

This means that we'll keep the following columns and just remove rows containing missing values for them: `emp_length`, `title`, `revol_util`, `last_credit_pull_d` and drop the `pub_rec_bankruptcies` column entirely.

In [17]:
loans = loans.drop("pub_rec_bankruptcies", axis=1)
loans = loans.dropna(axis=0)
print(loans.dtypes.value_counts())

object     11
float64    10
int64       1
dtype: int64


While the numerical columns can be used natively with scikit-learn, the object columns that contain text need to be converted to numerical data types. Let's return a new Dataframe containing just the object columns so we can explore them in more depth. 

In [18]:
object_columns_df = loans.select_dtypes(include=["object"])
print(object_columns_df.iloc[0])

term                     36 months
int_rate                    10.65%
emp_length               10+ years
home_ownership                RENT
verification_status       Verified
purpose                credit_card
title                     Computer
addr_state                      AZ
earliest_cr_line          Jan-1985
revol_util                   83.7%
last_credit_pull_d        Jun-2016
Name: 0, dtype: object


Some of the columns seem like they represent categorical values, but we should confirm by checking the number of unique values in those columns:

- `home_ownership`: home ownership status, can only be 1 of 4 categorical values according to the data dictionary
- `verification_status`: indicates if income was verified by Lending Club
- `emp_length`: number of years the borrower was employed upon time of application
- `term`: number of payments on the loan, either 36 or 60
- `addr_state`: borrower's state of residence
- `purpose`: a category provided by the borrower for the loan request
- `title`: loan title provided by the borrower

There are also some columns that represent numeric values, that need to be converted:

- `int_rate`: interest rate of the loan in %
- `revol_util`: revolving line utilization rate or the amount of credit the borrower is using relative to all available credit

Based on the first row's values for `purpose` and `title`, it seems like these columns could reflect the same information. Let's explore the unique value counts separately to confirm if this is true.

Lastly, some of the columns contain date values that would require a good amount of feature engineering for them to be potentially useful:

- `earliest_cr_line`: The month the borrower's earliest reported credit line was opened,
- `last_credit_pull_d`: The most recent month Lending Club pulled credit for this loan.

Since these date features require some feature engineering for modeling purposes, let's remove these date columns from the Dataframe.

## Checking categorical columns

In [20]:
cols = ['home_ownership', 'verification_status', 'emp_length', 'term', 'addr_state']

for col in cols:
    print(loans[col].value_counts())
    print("\n")

RENT        18112
MORTGAGE    16686
OWN          2778
OTHER          96
NONE            3
Name: home_ownership, dtype: int64


Not Verified       16281
Verified           11856
Source Verified     9538
Name: verification_status, dtype: int64


10+ years    8545
< 1 year     4513
2 years      4303
3 years      4022
4 years      3353
5 years      3202
1 year       3176
6 years      2177
7 years      1714
8 years      1442
9 years      1228
Name: emp_length, dtype: int64


 36 months    28234
 60 months     9441
Name: term, dtype: int64


CA    6776
NY    3614
FL    2704
TX    2613
NJ    1776
IL    1447
PA    1442
VA    1347
GA    1323
MA    1272
OH    1149
MD    1008
AZ     807
WA     788
CO     748
NC     729
CT     711
MI     678
MO     648
MN     581
NV     466
SC     454
WI     427
OR     422
LA     420
AL     420
KY     311
OK     285
KS     249
UT     249
AR     229
DC     209
RI     194
NM     180
WV     164
HI     162
NH     157
DE     110
MT      77
AK      76
WY      76
SD     

All 5 columns contain multiple discrete values. We should clean the emp_length column and treat it as a numerical one since the values have ordering (2 years of employment is less than 8 years).

First, let's look at the unique value counts for the `purpose` and `title` columns to understand which column we want to keep.

In [22]:
print(loans["title"].value_counts())
print("\n")
print(loans["purpose"].value_counts())

Debt Consolidation                          2068
Debt Consolidation Loan                     1599
Personal Loan                                624
Consolidation                                488
debt consolidation                           466
                                            ... 
Great Credit Individual Seeking Loan           1
Consolidate after home purchase & repair       1
big money                                      1
Windows keep the heat                          1
30 Year Roof Needs Replacing After 40          1
Name: title, Length: 18881, dtype: int64


debt_consolidation    17751
credit_card            4911
other                  3711
home_improvement       2808
major_purchase         2083
small_business         1719
car                    1459
wedding                 916
medical                 655
moving                  552
house                   356
vacation                348
educational             312
renewable_energy         94
Name: purpose, dtype: int6

The `home_ownership`, `verification_status`, `emp_length`, and `term` columns each contain a few discrete categorical values. We should encode these columns as dummy variables and keep them.

It seems like the `purpose` and `title` columns do contain overlapping information but we'll keep the `purpose` column since it contains a few discrete values. In addition, the `title` column has data quality issues since many of the values are repeated with slight modifications (e.g. `Debt Consolidation` and `Debt Consolidation Loan` and `debt consolidation`).

Lastly, the `addr_state` column contains many discrete values and we'd need to add 49 dummy variable columns to use it for classification. This would make our Dataframe much larger and could slow down how quickly the code runs. Let's remove this column from consideration.

In [23]:
# Drop unwanted columns, convert to numerical data type
mapping_dict = {
    "emp_length": {
        "10+ years": 10,
        "9 years": 9,
        "8 years": 8,
        "7 years": 7,
        "6 years": 6,
        "5 years": 5,
        "4 years": 4,
        "3 years": 3,
        "2 years": 2,
        "1 year": 1,
        "< 1 year": 0,
        "n/a": 0
    }
}

loans = loans.drop(["last_credit_pull_d", "addr_state", "title", "earliest_cr_line"], axis=1)

loans["int_rate"] = loans["int_rate"].str.rstrip("%").astype(float)
loans["revol_util"] = loans["revol_util"].str.rstrip("%").astype(float)

loans = loans.replace(mapping_dict)

In [24]:
# Convert to dummy columns
dummy_cols = ["home_ownership", "verification_status", "purpose", "term"]

dummy_df = pd.get_dummies(loans[dummy_cols])
loans = pd.concat([loans, dummy_df], axis=1)
loans = loans.drop(dummy_cols, axis=1)

In this section, we performed the last amount of data preparation necessary to start training machine learning models. We converted all of the columns to numerical values because those are the only type of value scikit-learn can work with. In the next section, we'll experiment with training models and evaluating accuracy using cross-validation.

# Making Predictions

Our eventual goal is to generate features from the data, which can feed into a machine learning algorithm. The algorithm will make predictions about whether or not a loan will be paid off on time, which is contained in the `loan_status` column of the clean dataset.

As we prepared the data, we removed columns that had data leakage issues, contained redundant information, or required additional processing to turn into useful features. We cleaned features that had formatting issues, and converted categorical columns to dummy variables.

We noticed that there's a class imbalance in our target column, `loan_status`. There are about 6 times as many loans that were paid off on time (positive case, label of 1) than those that weren't (negative case, label of 0). Imbalances can cause issues with many machine learning algorithms, where they appear to have high accuracy, but actually aren't learning from the training data. Because of its potential to cause issues, we need to keep the class imbalance in mind as we build machine learning models.

In [53]:
# The cleaned data from previous steps are stored in cleaned_loans_2007.csv
loans = pd.read_csv("cleaned_loans_2007.csv")
print(loans.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37675 entries, 0 to 37674
Data columns (total 38 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   loan_amnt                            37675 non-null  float64
 1   int_rate                             37675 non-null  float64
 2   installment                          37675 non-null  float64
 3   emp_length                           37675 non-null  int64  
 4   annual_inc                           37675 non-null  float64
 5   loan_status                          37675 non-null  int64  
 6   dti                                  37675 non-null  float64
 7   delinq_2yrs                          37675 non-null  float64
 8   inq_last_6mths                       37675 non-null  float64
 9   open_acc                             37675 non-null  float64
 10  pub_rec                              37675 non-null  float64
 11  revol_bal                   

## Picking an error metric

We established our question "Can we build a machine learning model that can accurately predict if a borrower will pay off their loan on time or not?" as a binary classification problem. Before diving in and selecting an algorithm to apply to the data, we should select an error metric.

An error metric will help us figure out when our model is performing well, and when it's performing poorly. To tie error metrics all the way back to the original question we wanted to answer, let's say we're using a machine learning model to predict whether or not we should fund a loan on the Lending Club platform. Our objective in this is to make money -- we want to fund enough loans that are paid off on time to offset our losses from loans that aren't paid off. An error metric will help us determine if our algorithm will make us money or lose us money.

In this case, we're primarily concerned with false positives and false negatives. Both of these are different types of misclassifications. With a false positive, we predict that a loan will be paid off on time, but it actually isn't. This costs us money, since we fund loans that lose us money. With a false negative, we predict that a loan won't be paid off on time, but it actually would be paid off on time. This loses us potential money, since we didn't fund a loan that actually would have been paid off.

Since we're viewing this problem from the standpoint of a conservative investor, we need to treat false positives differently than false negatives. A conservative investor would want to minimize risk, and avoid false positives as much as possible. They'd be more okay with missing out on opportunities (false negatives) than they would be with funding a risky loan (false positives).

We mentioned earlier that there is a significant class imbalance in the loan_status column. There are 6 times as many loans that were paid off on time (1), than loans that weren't paid off on time (0). This causes a major issue when we use accuracy as a metric. This is because due to the class imbalance, a classifier can predict 1 for every row, and still have high accuracy. In this case, we don't want to use accuracy, and should instead use metrics that tell us the number of false positives and false negatives.

This means that we should optimize for:

- high recall (true positive rate)
- low fall-out (false positive rate)

We can calculate false positive rate and true positive rate, using the numbers of true positives, true negatives, false negatives, and false positives.

False positive rate is the number of false positives divided by the number of false positives plus the number of true negatives. This divides all the cases where we thought a loan would be paid off but it wasn't by all the loans that weren't paid off: fpr = fp / (fp + tn)

True positive rate is the number of true positives divided by the number of true positives plus the number of false negatives. This divides all the cases where we thought a loan would be paid off and it was by all the loans that were paid off: tpr = tp / (tp + fn)

Simple english ways to think of each term are:

- False Positive Rate: "the percentage of the loans that shouldn't be funded that I would fund".
- True Positive Rate: "the percentage of loans that should be funded that I would fund".

Generally, if we want to reduce false positive rate, true positive rate will also go down. This is because if we want to reduce the risk of false positives, we wouldn't think about funding riskier loans in the first place.

## Picking machine learning models

As we saw in the first screen of the mission, our cleaned dataset contains 41 columns, all of which are either the int64 or the float64 data type. There aren't any null values in any of the columns. This means that we can now apply any machine learning algorithm to our dataset.

A good first algorithm to apply to binary classification problems is logistic regression, for the following reasons:

- it's quick to train and we can iterate more quickly,
- it's less prone to overfitting than more complex models like decision trees,
- it's easy to interpret.

In [54]:
cols = loans.columns
train_cols = cols.drop("loan_status")

features = loans[train_cols]
target = loans["loan_status"]

In [56]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict

# With class_weight set to balanced, scikit-learn will penalize the misclassification of the minority class during the training 
# process in order to combat class imbalances
lr = LogisticRegression(class_weight="balanced", max_iter=200)
# Make predictions using 3-fold cross-validation.
predictions = cross_val_predict(lr, features, target, cv=3)
predictions = pd.Series(predictions)

tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)

tp_num = len(predictions[tp_filter])
fp_num = len(predictions[fp_filter])
tn_num = len(predictions[tn_filter])
fn_num = len(predictions[fn_filter])

tpr = tp_num / (tp_num + fn_num)
fpr = fp_num / (fp_num + tn_num)

print(tpr)
print(fpr)

0.5418447624357307
0.31712748190758955


Our true positive rate is now around 54%, and our false positive rate is around 32%. From a conservative investor's standpoint, it's reassuring that the false positive rate is lower because it means that we'll be able to do a better job at avoiding bad loans than if we funded everything. However, we'd only ever decide to fund 54% of the total loans (true positive rate), so we'd immediately reject a good amount of loans.

We can try to lower the false positive rate further by assigning a harsher penalty for misclassifying the negative class. While setting class_weight to balanced will automatically set a penalty based on the number of 1s and 0s in the column, we can also set a manual penalty. In the last screen, the penalty scikit-learn imposed for misclassifying a 0 would have been around 5.89 (since there are 5.89 times as many 1s as 0s).

We can also specify a penalty manually if we want to adjust the rates more. To do this, we need to pass in a dictionary of penalty values to the class_weight parameter:

penalty = {0: 10, 1: 1}

lr = LogisticRegression(class_weight=penalty)

The above dictionary will impose a penalty of 10 for misclassifying a 0, and a penalty of 1 for misclassifying a 1.

In [57]:
penalty = {
    0: 10,
    1: 1
}

lr = LogisticRegression(class_weight=penalty, max_iter=200)

predictions = cross_val_predict(lr, features, target, cv=3)
predictions = pd.Series(predictions)

tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)

tp_num = len(predictions[tp_filter])
fp_num = len(predictions[fp_filter])
tn_num = len(predictions[tn_filter])
fn_num = len(predictions[fn_filter])

tpr = tp_num / (tp_num + fn_num)
fpr = fp_num / (fp_num + tn_num)

print(tpr)
print(fpr)

0.16750294245183672
0.06977175728335498


It looks like assigning manual penalties lowered the false positive rate to 7%, and thus lowered our risk. Note that this comes at the expense of true positive rate. While we have fewer false positives, we're also missing opportunities to fund more loans and potentially make more money. Given that we're approaching this as a conservative investor, this strategy makes sense, but it's worth keeping in mind the tradeoffs.

Let's try a more complex algorithm, random forest.  Random forests are able to work with nonlinear data, and learn complex conditionals. Logistic regressions are only able to work with linear data. Training a random forest algorithm may enable us to get more accuracy due to columns that correlate nonlinearly with `loan_status`.

In [58]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(class_weight="balanced", random_state=1)
predictions = cross_val_predict(rf, features, target, cv=3)
predictions = pd.Series(predictions)

tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)

tp_num = len(predictions[tp_filter])
fp_num = len(predictions[fp_filter])
tn_num = len(predictions[tn_filter])
fn_num = len(predictions[fn_filter])

tpr = tp_num / (tp_num + fn_num)
fpr = fp_num / (fp_num + tn_num)

print(tpr)
print(fpr)

0.9977079848850895
0.9918352198923733


Unfortunately, using a random forest classifier didn't improve our false positive rate. The model is likely weighting too heavily on the 1 class, and still mostly predicting 1s. We could fix this by applying a harsher penalty for misclassifications of 0s.

Ultimately, our best model had a false positive rate of nearly 7%, and a true positive rate of nearly 17%. For a conservative investor, this means that they make money as long as the interest rate is high enough to offset the losses from 7% of borrowers defaulting, and that the pool of 17% of borrowers is large enough to make enough interest money to offset the losses.

If we had randomly picked loans to fund, borrowers would have defaulted on 14.5% of them, and our model is better than that, although we're excluding more loans than a random strategy would.